In [26]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"


def df_preprocessing(df, grape_calculator_myfive, assign_area):
    print(f'データ前処理を行います')
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre = df_pre[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    df_pre["Grape_rate"] = grape_calculator_myfive(df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

    df_pre = df_pre.replace([np.inf, -np.inf], np.nan)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    print(f'以下のモデルが含まれています')
    for i, model in enumerate(model_list):
        print(f'{i}: {model}', end=", ")
    return df_pre



HALL_NAME = "EXA FIRST"
DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

# Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
df = df_preprocessing(df, grape_calculator_myfive, assign_area)
conn.close()
print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')

🔍 'EXA FIRST' を含むホール名が見つかりました。
データ前処理を行います
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, データサイズ: 60040 x 17


In [27]:
df.head()

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-05-02,ゴーゴージャグラー3,1001,5317,21,18,-68,253.2,295.4,6.32,136.3,2025-05,2,4,1,a
1,EXA FIRST,2025-05-02,ゴーゴージャグラー3,1002,635,2,1,-238,317.5,635.0,6.31,211.7,2025-05,2,4,2,a
2,EXA FIRST,2025-05-02,ゴーゴージャグラー3,1003,1386,4,4,-385,346.5,346.5,6.14,173.2,2025-05,2,4,3,a
3,EXA FIRST,2025-05-02,ゴーゴージャグラー3,1004,3288,13,14,332,252.9,234.9,6.17,121.8,2025-05,2,4,4,a
4,EXA FIRST,2025-05-02,ゴーゴージャグラー3,1005,894,2,1,-556,447.0,894.0,6.24,298.0,2025-05,2,4,5,a


## 出力データ
- 機種別の分析
- 台番号分析
- 日付分析
- 月分析
- 過去n日の差枚比較
- 全台系分析

In [69]:
today = datetime.date.today()
today = datetime.date(2024, 5, 6)

start_date = today - relativedelta(days=5)
end_date = today - relativedelta(days=0)
start_date, end_date

(datetime.date(2024, 5, 1), datetime.date(2024, 5, 6))

In [71]:

df_filtered = df.copy()
df_filtered = df_filtered[
    (df_filtered["date"].dt.date >= start_date)
    & (df_filtered["date"].dt.date <= end_date)
]

pivot_results = {}
pivot_targets = ["game", "medals", "BB", "RB"]
index_targets = ["area"]
for col in pivot_targets:
    table = df_filtered.pivot_table(
        index=index_targets,
        columns="date",
        values=col,
        aggfunc="sum",
        margins=True,
        margins_name="total",
    )
    pivot_results[col] = table.iloc[:, ::-1]

game = pivot_results["game"]
medals = pivot_results["medals"]
rb = pivot_results["RB"]
bb = pivot_results["BB"]
rb_rate = (game / rb).round(1)
total_rate = (game / (bb + rb)).round(1)

game = game.iloc[:, 1:]
medals = medals.iloc[:, 1:]
rb = rb.iloc[:, 1:]
bb = bb.iloc[:, 1:]
rb_rate = rb_rate.iloc[:, 1:]
total_rate = total_rate.iloc[:, 1:]

labeled_tables = [
    ("GAME", game),
    ("MEDALS", medals),
    ("RB_RATE", rb_rate),
    ("TOTAL_RATE", total_rate),
    ("BB", bb),
    ("RB", rb),
]

# ラベルを MultiIndex に付ける
for label, df_table in labeled_tables:
    df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

# 列を交互に整列して統合・NaN除去
interleaved_cols = [
    col
    for pair in zip(
        game.columns,
        medals.columns,
        bb.columns,
        rb.columns,
        rb_rate.columns,
        total_rate.columns,
    )
    for col in pair
]

merged = pd.concat([
    game,
    medals,
    bb,
    rb,
    rb_rate,
    total_rate
    ], axis=1)[interleaved_cols]
merged.to_csv(f"{HALL_NAME}_area_data.csv")
merged.head()

,GAME,MEDALS,BB,RB,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,...,BB,RB,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,RB_RATE,TOTAL_RATE
date,2024-05-06,2024-05-06,2024-05-06,2024-05-06,2024-05-06,2024-05-06,2024-05-05,2024-05-05,2024-05-05,2024-05-05,...,2024-05-02,2024-05-02,2024-05-02,2024-05-02,2024-05-01,2024-05-01,2024-05-01,2024-05-01,2024-05-01,2024-05-01
area,,,,,,,,,,,,,,,,,,,,,
a,63076,-2268,224,183,344.7,155.0,71717,4255,272,213,...,260,190,322.0,136.0,69408,9112,283,230,301.8,135.3
b,68325,11660,288,227,301.0,132.7,74738,-914,258,258,...,250,217,316.3,147.0,57416,-4753,205,150,382.8,161.7
c,35553,742,133,97,366.5,154.6,44696,1853,166,139,...,190,128,336.2,135.3,43409,7120,179,145,299.4,134.0
d,33537,-1805,114,104,322.5,153.8,43580,3335,174,133,...,133,97,351.5,148.3,42374,3167,157,151,280.6,137.6
e,51589,353,192,149,346.2,151.3,69680,13963,300,237,...,223,200,294.5,139.3,65829,13636,286,214,307.6,131.7


In [4]:
# 各種ピボットテーブル
def create_pivot_tabel(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    # df_filtered = df_filtered[df_filtered["area"] == area]
    df_filtered = df_filtered[(df_filtered["date"].dt.date >= start_date) & (df_filtered["date"].dt.date <= end_date)]
    pivot_targets = ["game", "medals", "RB_rate", "Total_rate", "BB", "RB"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
        index=["model_name", 'area', "unit_no"],
        columns="date",
        values=col,
        aggfunc="sum",
    )
        # 日付列を反転・スライス
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("BB", bb),
        ("RB", rb),
        ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols = [col for pair in zip(
        game.columns,
        medals.columns,
        bb.columns,
        rb.columns,
        rb_rate.columns,
        total_rate.columns,
    ) for col in pair]
    
    merged = pd.concat([game, medals, bb, rb, rb_rate, total_rate], axis=1)[interleaved_cols]
    
    return merged


end_date = datetime.date.today()
start_date =  end_date - relativedelta(days=3)
merged = create_pivot_tabel(df, "マイジャグラーV", start_date, end_date)
merged.head()

GAME     MEDALS         BB         RB  \
date                    2025-05-02 2025-05-02 2025-05-02 2025-05-02   
model_name area unit_no                                               
マイジャグラーV   c    1021          6785      -1453         18         21   
                1022          2233      -1076          4          6   
                1023          3522       -756         11          8   
                1024          2461       -759          7          6   
                1025          1178      -1088          2          0   

                           RB_RATE TOTAL_RATE       GAME     MEDALS  \
date                    2025-05-02 2025-05-02 2025-05-01 2025-05-01   
model_name area unit_no                                               
マイジャグラーV   c    1021         323.1      174.0       6803      -1229   
                1022         372.2      223.3       8704       5206   
                1023         440.2      185.4       7751       1488   
                1024         410.2      189.3       6661       1544   
                1025           0.0      589.0       7689       2268   

                                BB         RB    RB_RATE TOTAL_RATE  \
date                    2025-05-01 2025-05-01 2025-05-01 2025-05-01   
model_name area unit_no                                               
マイジャグラーV   c    1021            20         22      309.2      162.0   
                1022            49         36      241.8      102.4   
                1023            33         24      323.0      136.0   
                1024            27         24      277.5      130.6   
                1025            35         24      320.4      130.3   

                              GAME     MEDALS         BB         RB  \
date                    2025-04-30 2025-04-30 2025-04-30 2025-04-30   
model_name area unit_no                                               
マイジャグラーV   c    1021          6977      -2603         14         23   
                1022          5396       -903         19         12   
                1023           927       -456          2          2   
                1024          7662       1738         31         31   
                1025          3007       -468         10          9   

                           RB_RATE TOTAL_RATE  
date                    2025-04-30 2025-04-30  
model_name area unit_no                        
マイジャグラーV   c    1021         303.3      188.6  
                1022         449.7      174.1  
                1023         463.5      231.8  
                1024         247.2      123.6  
                1025         334.1      158.3

In [ ]:
model_list = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
    "ミスタージャグラー",
    "ウルトラミラクルジャグラー",
    "ハッピージャグラーVIII",
    "ジャグラーガールズ",
]

end_date = datetime.date.today()
start_date =  end_date - relativedelta(days=30)

merged_models = pd.DataFrame()
area_summary = pd.DataFrame()
for model in list(model_list):
    print(model)
    model_merged = create_pivot_tabel(df, model, start_date, end_date)
    for area in model_merged.index.get_level_values("area").unique():
        area_merged = model_merged.xs(area, level="area", drop_level=False)
        # area_merged = area_merged[~area_merged.iloc[:, 0].isna()]
        empty_index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
        empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
        
        if area_merged.shape[0]:
            summary_row = []
            for c in area_merged.columns:
                if c[0] in ["GAME", "MEDALS", "RB", "BB"]:
                    game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
                    summary_row.append(area_merged[c].sum())
                elif c[0] == "RB_RATE":
                    game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
                    rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
                    rb_rate = game_sum / rb_sum if rb_sum != 0 else None
                    summary_row.append(round(rb_rate, 1) if rb_rate else None)
                elif c[0] == "TOTAL_RATE":
                    game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
                    rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
                    bb_sum = area_merged.get(("BB", c[1]), pd.Series()).sum()
                    total_rate = game_sum / (rb_sum + bb_sum) if (rb_sum + bb_sum) != 0 else None
                    summary_row.append(round(total_rate, 1) if total_rate else None)
            

            summary_df = pd.DataFrame(summary_row).T
            summary_df.index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
            summary_df.columns = area_merged.columns
            
            area_summary = pd.concat([area_summary, summary_df])
            
            merged_models = pd.concat([merged_models, area_merged, summary_df, empty_row])

merged_models = merged_models[~(merged_models.iloc[:, 0].isna() | (merged_models.iloc[:, 0] == 0))]
area_summary.to_csv(f"{HALL_NAME}_area_summary.csv")
merged_models.to_csv(f"{HALL_NAME}_data.csv")
    
merged_models.tail(10)

マイジャグラーV


ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

In [6]:
len(area_merged.columns)

180

In [15]:
summary_row = []

for c in area_merged.columns:
    if c[0] in ["GAME", "MEDALS", "RB", "BB"]:
        game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        summary_row.append(area_merged[c].sum())
    
    elif c[0] == "RB_RATE":
        game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        rb_rate = game_sum / rb_sum if rb_sum != 0 else None
        summary_row.append(round(rb_rate, 1) if rb_rate else None)

    elif c[0] == "TOTAL_RATE":
        game_sum = area_merged.get(("GAME", c[1]), pd.Series()).sum()
        rb_sum = area_merged.get(("RB", c[1]), pd.Series()).sum()
        bb_sum = area_merged.get(("BB", c[1]), pd.Series()).sum()
        total_rate = game_sum / (rb_sum + bb_sum) if (rb_sum + bb_sum) != 0 else None
        summary_row.append(round(total_rate, 1) if total_rate else None)

summary_df = pd.DataFrame(summary_row).T
summary_df.index = pd.MultiIndex.from_tuples([tuple([""] * area_merged.index.nlevels)], names=area_merged.index.names)
summary_df.columns = area_merged.columns
summary_df

,,,GAME,MEDALS,BB,RB,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,...,BB,RB,RB_RATE,TOTAL_RATE,GAME,MEDALS,BB,RB,RB_RATE,TOTAL_RATE
,,date,2025-05-02,2025-05-02,2025-05-02,2025-05-02,2025-05-02,2025-05-02,2025-05-01,2025-05-01,2025-05-01,2025-05-01,...,2025-04-04,2025-04-04,2025-04-04,2025-04-04,2025-04-03,2025-04-03,2025-04-03,2025-04-03,2025-04-03,2025-04-03
model_name,area,unit_no,,,,,,,,,,,,,,,,,,,,,
,,,10907.0,-677.0,43.0,19.0,574.1,175.9,21639.0,-2586.0,80.0,44.0,...,27.0,23.0,425.9,195.9,20283.0,-1562.0,68.0,58.0,349.7,161.0


In [8]:
spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[HALL_NAME]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)